# Emotion classification with BERT

This notebook focuses on classifying text with emotions using the BERT transformer model. The goal is to analyze and understand the emotions conveyed in textual data.

By Constant Fotie Moghommahie

[fotiecodes](https://fotiecodes.com).


Classes: 
```json {
    'sadness': 0,
    'fear': 1,
    'anger': 2,
    'love': 3,
    'happy': 4,
    'surprise': 5
}
```

### 1. Import and install neccessary libs


In [10]:
!pip3 install nltk
!pip3 install --upgrade tensorflow



[notice] A new release of pip available: 22.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [11]:
import tensorflow as tf
#import tensorflow_hub as hub
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import re
import unicodedata
from nltk.corpus import stopwords
from tensorflow import keras
from tensorflow.keras.layers import Dense,Dropout, Input
#from tqdm import tqdm
import pickle
from sklearn.metrics import confusion_matrix,f1_score,classification_report
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from tensorflow.keras import regularizers

In [12]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fotiem.constant/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
from transformers import BertTokenizer, TFBertModel, BertConfig,TFDistilBertModel,DistilBertTokenizer,DistilBertConfig

### 2.  Preprocessing and cleaning functions

In [14]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def clean_stopwords_shortwords(w):
    stopwords_list=stopwords.words('english')
    words = w.split() 
    clean_words = [word for word in words if (word not in stopwords_list) and len(word) > 2]
    return " ".join(clean_words) 

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.!,¿])", r" ", w)
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    w=clean_stopwords_shortwords(w)
    w=re.sub(r'@\w+', '',w)
    return w

### 3. Load and Inspect our dataset
We will first load and and preview our dataset. So we can check for any missing values.

In [76]:
# Load and inspect the new dataset
data_file = './data/emotion_final.csv'

# Read the data from the CSV file into a pandas DataFrame
data=pd.read_csv(data_file,encoding='ISO-8859-1')

# Display the first few rows of the DataFrame
data.head()

,Text,Emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


>  Removing Unnamed Columns, dropping NaN data and resetting the index after dropping some rows/columns containing NaN dataset and finally shuffling the dataset

In [77]:
print('File has {} rows and {} columns'.format(data.shape[0],data.shape[1]))
data=data.dropna()
data=data.reset_index(drop=True)
print('File has {} rows and {} columns'.format(data.shape[0],data.shape[1]))
data = shuffle(data)

data.head()

File has 21459 rows and 2 columns
File has 21459 rows and 2 columns


,Text,Emotion
16084,i came home waiting for the shower read someth...,sadness
8328,i am mostly feeling contentedly terrified abou...,fear
13309,i cant help but feel someones going to end up ...,anger
933,im feeling low and forgotten,sadness
7336,i was like that i always wanted to feel and be...,love


In [106]:
data=data.rename(columns = {'Emotion': 'label', 'Text': 'text'}, inplace = False)

# Ensure all unique labels are included in the mapping dictionary
unique_labels = data['label'].unique()
mapping_dict = {label: index for index, label in enumerate(unique_labels)}

data['gt'] = data['label'].map(mapping_dict)

# Fill NaN values with a specific value (e.g., -1) and convert to integer
data['gt'] = data['gt'].fillna(-1).astype(int)

data['text']=data['text'].map(preprocess_sentence)

num_classes=len(data.label.unique())

data.head()

,text,label,gt
16084,came home waiting shower read something made u...,sadness,0
8328,mostly feeling contentedly terrified,fear,1
13309,cant help feel someones going end pissed,anger,2
933,feeling low forgotten,sadness,0
7336,like always wanted feel accepted family others,love,3


###  4. Loading DistilBERT Tokenizer and the DistilBERT model

In [107]:
dbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [108]:
dbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_transform', 'vocab_projector', 'activation_13', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


###  5. Preparing input for the model

In [109]:
# Set the maximum length of the input sentences
max_len = 32

# Get the sentences and labels from the data
sentences = data['text']
labels = data['gt']

# Print the length of the sentences and labels
len(sentences), len(labels)


(21459, 21459)

####  Let's take a sentence from the dataset and understand the input and output of the DistilBERT

In [110]:
sentences[0]

'didnt feel humiliated'

In [111]:
dbert_tokenizer.tokenize(sentences[0])

['didn', '##t', 'feel', 'humiliated']

> Input ids and the attention masks from the tokenizer

In [112]:
dbert_inp=dbert_tokenizer.encode_plus(sentences[0],add_special_tokens = True,max_length =20,pad_to_max_length = True,truncation=True)
dbert_inp

/Users/fotiem.constant/.pyenv/versions/3.10.6/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2285: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


{'input_ids': [101, 2134, 2102, 2514, 26608, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [113]:
dbert_inp['input_ids']

[101, 2134, 2102, 2514, 26608, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

>  DistilBERT model output: Give input_ids and the attention_mask obtained from the tokenizer. The output will be a tuple of the size (1,max_len,768)

In [114]:
id_inp=np.asarray(dbert_inp['input_ids'])
mask_inp=np.asarray(dbert_inp['attention_mask'])
out=dbert_model([id_inp.reshape(1,-1),mask_inp.reshape(1,-1)])
type(out),out

(transformers.modeling_tf_outputs.TFBaseModelOutput,
 TFBaseModelOutput(last_hidden_state=<tf.Tensor: shape=(1, 20, 768), dtype=float32, numpy=
 array([[[-0.17138529,  0.04808471, -0.11152925, ..., -0.06434252,
           0.330923  ,  0.3642112 ],
         [ 0.35398188,  0.23759897,  0.2169984 , ...,  0.00849794,
           0.46305764, -0.41722077],
         [-0.4177962 ,  0.0801597 ,  0.4836896 , ..., -0.17692414,
           0.5626385 ,  0.43658397],
         ...,
         [-0.15297796,  0.03547437,  0.06371675, ...,  0.03469756,
           0.02733814, -0.16189006],
         [-0.14419688,  0.06348484,  0.07919805, ...,  0.11205959,
           0.04838258, -0.20951843],
         [-0.1142126 ,  0.06052709,  0.07216727, ...,  0.12287258,
           0.0578858 , -0.19464062]]], dtype=float32)>, hidden_states=None, attentions=None))

> Obtain the embeddings of a sentence from the output

In [115]:
out[0][:,0,:]

<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
array([[-1.71385288e-01,  4.80847135e-02, -1.11529246e-01,
        -2.09110361e-02, -2.34398529e-01, -4.43109423e-02,
         2.90086329e-01,  2.66656995e-01,  2.05920771e-01,
        -2.20593363e-01,  5.82960322e-02, -2.02091962e-01,
        -1.94992989e-01,  1.50783703e-01,  9.85642374e-02,
         5.78699633e-02,  2.98273973e-02,  1.51283860e-01,
         6.53597992e-03, -5.72893135e-02, -7.78305829e-02,
        -3.38845789e-01, -1.36250705e-01,  7.62559399e-02,
        -1.06427193e-01,  3.59944515e-02,  1.48689240e-01,
        -3.07668000e-01,  6.57812208e-02, -1.10575132e-01,
         6.58358634e-02,  2.32130259e-01, -1.61469787e-01,
         2.03128271e-02, -1.84822604e-01,  1.77118883e-01,
         3.92266028e-02,  1.18414506e-01,  1.43307328e-01,
         1.92268342e-02, -2.72444606e-01, -5.23971878e-02,
        -2.44558007e-01, -4.83159050e-02,  1.09537043e-01,
        -1.48615271e-01, -2.21261787e+00, -3.53778973e-02,
      

> Decode the original sentence from the tokenizer 

In [116]:
dbert_tokenizer.decode(dbert_inp['input_ids'])

'[CLS] didnt feel humiliated [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [117]:
print("Available labels: ",data.label.unique())
num_classes = len(data.label.unique())
data.head()

Available labels:  ['sadness' 'fear' 'anger' 'love' 'happy' 'surprise']


,text,label,gt
16084,came home waiting shower read something made u...,sadness,0
8328,mostly feeling contentedly terrified,fear,1
13309,cant help feel someones going end pissed,anger,2
933,feeling low forgotten,sadness,0
7336,like always wanted feel accepted family others,love,3


###  6. Create a basic NN model using DistilBERT embeddings to get the predictions

In [118]:
def create_model():
    inps = Input(shape = (max_len,), dtype='int64')
    masks= Input(shape = (max_len,), dtype='int64')
    dbert_layer = dbert_model(inps, attention_mask=masks)[0][:,0,:]
    dense = Dense(512,activation='relu',kernel_regularizer=regularizers.l2(0.01))(dbert_layer)
    dropout= Dropout(0.5)(dense)
    pred = Dense(num_classes, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout)
    model = tf.keras.Model(inputs=[inps,masks], outputs=pred)
    print(model.summary())
    return model   

> Feel free to add more Dense and Dropout layers with variable units and the regularizers

In [119]:
model=create_model()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 32)]                 0         []                            
                                                                                                  
 input_6 (InputLayer)        [(None, 32)]                 0         []                            
                                                                                                  
 tf_distil_bert_model_5 (TF  TFBaseModelOutput(last_hid   6636288   ['input_5[0][0]',             
 DistilBertModel)            den_state=(None, 32, 768),   0          'input_6[0][0]']             
                              hidden_states=None, atten                                           
                             tions=None)                                                    

> Prepare the model input

In [120]:
input_ids=[]
attention_masks=[]

for sent in sentences:
    dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
    input_ids.append(dbert_inps['input_ids'])
    attention_masks.append(dbert_inps['attention_mask'])

input_ids=np.asarray(input_ids)
attention_masks=np.array(attention_masks)
labels=np.array(labels)

/Users/fotiem.constant/.pyenv/versions/3.10.6/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2285: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [121]:
len(input_ids),len(attention_masks),len(labels)

(21459, 21459, 21459)

> Save the model input in the pickle files to use it later without performing the above steps

In [122]:
print('Preparing the pickle file.....')

pickle_inp_path='./data/pickle_files/dbert_inp.pkl'
pickle_mask_path='./data/pickle_files/dbert_mask.pkl'
pickle_label_path='./data/pickle_files/dbert_label.pkl'

Preparing the pickle file.....


In [123]:
pickle.dump((input_ids),open(pickle_inp_path,'wb'))
pickle.dump((attention_masks),open(pickle_mask_path,'wb'))
pickle.dump((labels),open(pickle_label_path,'wb'))


print('Pickle files saved as ',pickle_inp_path,pickle_mask_path,pickle_label_path)

Pickle files saved as  ./data/pickle_files/dbert_inp.pkl ./data/pickle_files/dbert_mask.pkl ./data/pickle_files/dbert_label.pkl


In [124]:
print('Loading the saved pickle files..')

input_ids=pickle.load(open(pickle_inp_path, 'rb'))
attention_masks=pickle.load(open(pickle_mask_path, 'rb'))
labels=pickle.load(open(pickle_label_path, 'rb'))

print('Input shape {} Attention mask shape {} Input label shape {}'.format(input_ids.shape,attention_masks.shape,labels.shape))

Loading the saved pickle files..
Input shape (21459, 32) Attention mask shape (21459, 32) Input label shape (21459,)


In [126]:
label_class_dict = dict(enumerate(data['label'].unique()))
target_names = label_class_dict.values()
target_names


dict_values(['sadness', 'fear', 'anger', 'love', 'happy', 'surprise'])

> Train Test split and setting up the loss function, accuracy and optimizer for the model. 

In [127]:
train_inp,val_inp,train_label,val_label,train_mask,val_mask=train_test_split(input_ids,labels,attention_masks,test_size=0.2)

print('Train inp shape {} Val input shape {}\nTrain label shape {} Val label shape {}\nTrain attention mask shape {} Val attention mask shape {}'.format(train_inp.shape,val_inp.shape,train_label.shape,val_label.shape,train_mask.shape,val_mask.shape))


log_dir='dbert_model'
model_save_path='./dbert_model.h5'

callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

model.compile(loss=loss,optimizer=optimizer, metrics=[metric])

Train inp shape (17167, 32) Val input shape (4292, 32)
Train label shape (17167,) Val label shape (4292,)
Train attention mask shape (17167, 32) Val attention mask shape (4292, 32)


In [128]:
callbacks= [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]
model.compile(loss=loss,optimizer=optimizer, metrics=[metric])

### Training

In [129]:
history=model.fit([train_inp,train_mask],train_label,batch_size=16,epochs=1,validation_data=([val_inp,val_mask],val_label),callbacks=callbacks)

/Users/fotiem.constant/.pyenv/versions/3.10.6/lib/python3.10/site-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1073/1073 [==============================] - 5748s 5s/step - loss: 5.4224 - accuracy: 0.8374 - val_loss: 4.0729 - val_accuracy: 0.9271


### Tensorboard visualization (Training-Testing curve)

In [130]:
%load_ext tensorboard

In [131]:
%tensorboard --logdir {log_dir}

### We will Increase the number of epochs in order to decrease the loss further
Here we will use the saved model for predictions and calculating the evaluation metrics

In [132]:
trained_model = create_model()
trained_model.compile(loss=loss,optimizer=optimizer, metrics=[metric])
trained_model.load_weights(model_save_path)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 32)]                 0         []                            
                                                                                                  
 input_8 (InputLayer)        [(None, 32)]                 0         []                            
                                                                                                  
 tf_distil_bert_model_5 (TF  TFBaseModelOutput(last_hid   6636288   ['input_7[0][0]',             
 DistilBertModel)            den_state=(None, 32, 768),   0          'input_8[0][0]']             
                              hidden_states=None, atten                                           
                             tions=None)                                                    

In [133]:
preds = trained_model.predict([val_inp,val_mask],batch_size=16)
pred_labels = preds.argmax(axis=1)
f1 = f1_score(val_label, pred_labels, average='weighted')
f1

269/269 [==============================] - 248s 905ms/step


0.9264214263864357

In [134]:
print(target_names)
# print(target_names.shape)
print(val_label.shape)
print(pred_labels.shape)


# we print F1 score and classification report
print('F1 score:', f1)
print('Classification Report:')
print(classification_report(val_label, pred_labels))

print('Training and saving built model...')

dict_values(['sadness', 'fear', 'anger', 'love', 'happy', 'surprise'])
(4292,)
(4292,)
F1 score: 0.9264214263864357
Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.98      0.96      1228
           1       0.83      0.91      0.87       492
           2       0.94      0.91      0.92       627
           3       0.85      0.87      0.86       328
           4       0.96      0.95      0.95      1423
           5       0.99      0.66      0.80       194

    accuracy                           0.93      4292
   macro avg       0.92      0.88      0.89      4292
weighted avg       0.93      0.93      0.93      4292

Training and saving built model...


### Quick test in real world scenario
We are now gonna try to predict a sentence with any input

In [145]:
# now we try to predict the label of a random sentence from user
def predict(sentence):
    # we first preprocess the sentence
    sentence = preprocess_sentence(sentence)
    # then we do some tokenization on the sentence
    dbert_inps=dbert_tokenizer.encode_plus(sentence,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
    # then we convert to numpy array
    id_inp=np.asarray(dbert_inps['input_ids'])
    mask_inp=np.asarray(dbert_inps['attention_mask'])
    # and predict the label using the trained model
    pred = trained_model.predict([id_inp.reshape(1,-1),mask_inp.reshape(1,-1)])
    # and then get the label with the highest probability
    pred_label = np.argmax(pred,axis=1)
    # we then return it
    return label_class_dict[pred_label[0]]

In [146]:
# test the model on a random sentence
predict("I was alone at home and now i have no idea what i will be doing for the next 2 weeks")

/Users/fotiem.constant/.pyenv/versions/3.10.6/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2285: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


1/1 [==============================] - 1s 1s/step


'sadness'